##  2.1 Real-time Data 

### 2.1.1  setup Tags of source points in DataBase

In [ ]:
# -*- coding: utf-8 -*-
import redis

conn = redis.Redis('localhost')

def TagDefToRedisHashKey(tagdeflist):
    pipe = conn.pipeline()
    for element in tagdeflist:
        pipe.hmset(element['id'], 
                   {'desc': element['desc'], 'value': "-10000", 'ts': ""})
    pipe.execute()

def TagDefFromRedisHash(tagdeflist):
    taglist = []
    for element in tagdeflist:
        htag = conn.hgetall(element['id'])
        taglist.append(htag)
    return taglist


In [ ]:
unittagdeflist=[]
unittagdeflist.append({'id':"CSDC.DCS2AI.2JZA2226",'desc':"二号机组高压缸入口压力"})
unittagdeflist.append({'id':"CSDC.DCS2AI.2JZA2408",'desc':"二号机组高压缸入口温度"})
unittagdeflist.append({'id':"CSDC.DCS2AI.2JZA2230",'desc':"二号机组高压缸出口压力"})
unittagdeflist.append({'id':"CSDC.DCS2AI.2JZA2459",'desc':"二号机组高压缸出口温度"})
unittagdeflist.append({'id':"CSDC.DCS2AI.2JZA2264",'desc':"大气压力"})

TagDefToRedisHashKey(unittagdeflist)

taglist=TagDefFromRedisHash(unittagdeflist)
print(taglist[0])
print(taglist[0][b'desc'].decode())

### 2.1.2 sampling and send data to DataBase

In [ ]:
# -*- coding: utf-8 -*-
from datetime import *

def SendToRedisHash(tagvaluelist):
    pipe = conn.pipeline()

    curtime = datetime.now()

    for element in tagvaluelist:
        pipe.hmset(element['id'], 
                   {'value': element['value'], 'ts': curtime})
    pipe.execute()


In [ ]:
unittagvaluelist=[]
unittagvaluelist.append({'id':"CSDC.DCS2AI.2JZA2226",'value':16.38525})
unittagvaluelist.append({'id':"CSDC.DCS2AI.2JZA2408",'value':538.37})
unittagvaluelist.append({'id':"CSDC.DCS2AI.2JZA2230",'value':3.035039})
unittagvaluelist.append({'id':"CSDC.DCS2AI.2JZA2459",'value':313.2931})
unittagvaluelist.append({'id':"CSDC.DCS2AI.2JZA2264",'value':100.5281})

SendToRedisHash(unittagvaluelist)

print(unittagvaluelist[0])
# get back from redis
data=conn.hmget(unittagvaluelist[0]['id'], 'value', 'ts')
print(data[0].decode(),data[1].decode())


## 2.2 online analysis

### 2.2.1 setup Tags of results in  DataBase

In [ ]:
# 1 setup Tags of results in DataBase
unit_outtaglist=[]
unit_outtaglist.append({'id':"DEMO.DCS2AO.HP_EFF_IS",'desc':"二号机组高压缸效率"})

TagDefToRedisHashKey(unit_outtaglist)

### 2.2.2 Get Real-time Data from Redis

In [ ]:
# 2 get real-time data
# -*- coding: utf-8 -*-
def tagvalue_redis(taglist):
  
    tagcount=len(taglist)
    
    pipe = conn.pipeline()
    for i in range(tagcount):
        pipe.hmget(taglist[i]['id'], 'value', 'ts')
    tagvaluelist = pipe.execute()

    for i in range(tagcount):
        taglist[i]['value'] = tagvaluelist[i][0].decode()
        taglist[i]['ts'] = tagvaluelist[i][1].decode()

unittaglist=[]
unittaglist.append({'id':"CSDC.DCS2AI.2JZA2226"})
unittaglist.append({'id':"CSDC.DCS2AI.2JZA2408"})
unittaglist.append({'id':"CSDC.DCS2AI.2JZA2230"})
unittaglist.append({'id':"CSDC.DCS2AI.2JZA2459"})
unittaglist.append({'id':"CSDC.DCS2AI.2JZA2264"})

tagvalue_redis(unittaglist)

print(unittaglist[4])

pam=float(unittaglist[4]['value'])/1000

### 2.2.3 analyze

In [ ]:
# -*- coding: utf-8 -*-
# 3 analysis

from seuif97 import *

def CylinderEff(cylinder):
    """simple function  for cylinde using 'dict' """

    cylinder['inlet']['h'] = pt2h(cylinder['inlet']['p'], cylinder['inlet']['t'])
    cylinder['inlet']['s'] = pt2s(cylinder['inlet']['p'], cylinder['inlet']['t'])

    cylinder['outlet']['h'] = pt2h(cylinder['outlet']['p'], cylinder['outlet']['t'])
    cylinder['outlet']['s'] = pt2s(cylinder['outlet']['p'], cylinder['outlet']['t'])

    # h2s is the specific enthalpy at state 2 for the isentropic turbine
    h2s = ps2h(cylinder['outlet']['p'], cylinder['inlet']['s'])
    
    cylinder['h2s'] = h2s
     
    hds = cylinder['inlet']['h'] - h2s  # isentropic specific enthalpy drop
    hd = cylinder['inlet']['h'] - cylinder['outlet']['h']  # specific enthalpy drop

    cylinder['eff'] = 100 * hd / hds

    return cylinder

hp = {'inlet':{}, 'outlet':{}, 'h2s':None,'eff':None}

minlet = {'p':None, 't': None, 'h': None, 's':None}
moutlet = {'p': None, 't': None, 'h': None, 's': None}

minlet['p']=float(unittaglist[0]['value'])+pam
minlet['t']=float(unittaglist[1]['value'])
moutlet['p']=float(unittaglist[2]['value'])+pam
moutlet['t']=float(unittaglist[3]['value'])

hp['inlet'] = dict(minlet)
hp['outlet'] = dict(moutlet)

hp = CylinderEff(hp)

print('The internal efficiency =', hp['eff'], '%')

### 2.2.4 send results  to database

In [ ]:
# 4 send result to database--------------------------
unit_outtaglist[0]['value']=hp['eff']

SendToRedisHash(unit_outtaglist)

# get back from redis
taglist=TagDefFromRedisHash(unit_outtaglist)
print(taglist[0])
print(taglist[0][b'desc'].decode(),taglist[0][b'value'].decode())


## Reference

<ol>
<li>The Redis Documentation. http://redis.io/documentation
<li>JOSIAH L. CARLSON. Redis in Action. Manning Publications Co. 2013 
</ol>